In [1]:
import sys
sys.path.append('../src')
from preprocess import data

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12789 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12789 non-null  object 
 1   availability  12789 non-null  object 
 2   location      12789 non-null  object 
 3   size          12789 non-null  object 
 4   society       12789 non-null  object 
 5   total_sqft    12789 non-null  object 
 6   price         12789 non-null  float64
 7   bath          12789 non-null  int32  
 8   balcony       12789 non-null  int32  
dtypes: float64(1), int32(2), object(6)
memory usage: 899.2+ KB
